In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 

In [5]:
#set visulization style 

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12,6)
print("libraries loaded ")

libraries loaded 
